In [80]:
!pip install aifactory
!pip install gradio
!pip install openai
!pip install wget
!Pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 모델 추론 함수

In [85]:
import numpy as np
from PIL import Image
from skimage import io
import plotly.express as px
import plotly.graph_objects as go
import requests

import json
import numpy as np

def predict_image(img_input, messages):

    url = "http://34.168.13.204:8000/predict"

    print(img_input)
    dfile = open(img_input, "rb")
    test_res = requests.post(url, files = {"file": dfile})
    dfile.close()

    if test_res.ok != True:
        state = [(None, "고서한자 이미지가 제대로 업로드 되지 않았습니다.")]
        return None, state, messages 

    preds = json.loads(test_res.text) # string to dict

    img = np.array(Image.open(img_input))
    img_output = px.imshow(img)
    img_output.update_xaxes(visible=False)
    img_output.update_yaxes(visible=False)
    img_output.update_layout(coloraxis_showscale=False, height=800)

    labels = []
    chars = []

    for pred in list(preds.values())[0]:
        label = pred['label'] # Pred label
        
        x1, y1, w, h = [int(x) for x in pred['bbox']]

        chars.append((x1, y1, label))

        labels.append(label + "(" + str(x1) +"," + str(y1) + ")")

        x2, y2 = int(x1+w), int(y1+h)
        box = np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2],[x1,y1]])
        # Draw each bbox
        img_output.add_trace(
            go.Scatter(
            x=box[:,0], y=box[:,1], mode='lines', line_color='red', name=label, opacity=0.7, showlegend=False))

    chars.sort()

    # 출력 항목을 고서 읽는 방향으로 정렬시킵니다.

    sorted_chars = []
    seed = chars[0][0]
    index_count = 99

    for c in chars:
        calc = c[0] - seed
        if calc > 100:
            index_count -= 1
            seed = c[0]
        sorted_chars.append((index_count, c[1], c[2]))

    sorted_chars.sort()

    # 정렬된 항목에서 라벨값만 추출하여 문장을 구성합니다.

    sorted_sentence = ""
    for sc in sorted_chars:
        sorted_sentence += sc[2]
        
    messages.append({"role" : "system", "content": "다음의 조선시대 고서 원문 한자를 원문과 비교하여 한 문장씩 한글로 풀어서 쉽게 변역합니다. 이어서 종합적인 해석문을 작성합니다."})
    messages.append({"role" : "user", "content": "원문: " + f"{sorted_sentence}"})
    
    import openai

    openai.api_key = "OpenAI API Key" # 수정필요
                                
    res = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

    messages.append({"role" : "assistant", "content": res['choices'][0]['message']['content']})

    chatbot = []

    for msg in messages:
        if msg["role"] == "user":
            chatbot.append((msg["content"], None))
        elif msg["role"] == "assistant":
            chatbot.append((None, msg["content"]))

    return img_output, chatbot, messages

def predict_chat(txt_input, messages):

    import openai

    messages.append({"role" : "user", "content": f"{txt_input}\n"})

    openai.api_key = "OpenAI API Key" # 수정필요
                                
    res = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

    messages.append({"role" : "assistant", "content": res['choices'][0]['message']['content']})

    chatbot = []

    for msg in messages:
        if msg["role"] == "user":
            chatbot.append((msg["content"], None))
        elif msg["role"] == "assistant":
            chatbot.append((None, msg["content"]))

    return chatbot, messages

def gui_interface(predict_image_func, predict_chat_func):

    import gradio as gr
    import wget
    import os

    exam_imgs = ['example_1.jpg', 'example_2.jpg', 'example_3.jpg']

    for img_item in exam_imgs:
        wget.download("https://raw.githubusercontent.com/aifactory-team/AFCompetition/main/2302/" + img_item,
                      os.getcwd() + "/" + img_item)

    with gr.Blocks() as demo:
        
        # define components
        state = gr.State([])

        gr.Markdown(
        """
        # 고서이미지번역GPT
        고서 이미지를 업로드 한 후 [분석요청] 버튼을 누르시면, OCR AI 모델이 한자를 추출합니다. 그리고 챗GPT가 추출한 한자 문장을 한국어로 번역해드립니다. 그 후에는 챗GPT와 자유롭게 내용에 대해 대화를 나누실 수 있습니다.
        
        ### 참고사항
        * 고서 이미지를 분석하는 시간이 80~90초 정도 걸리니 [분석요청] 버튼 클릭 후 잠시 기다려주시면 감사하겠습니다.
        * 챗GPT의 고서한자 번역 성능이 검증되지 않았으며, 고서한자 번역한 내용이 정확하지 않을 수 있습니다. 
        """)

        with gr.Row():
            with gr.Column(scale=0.5):
                img_input = gr.Image(type='filepath', label='Input Image')
                btn_analysis = gr.Button("분석요청")
                examples = gr.Examples(examples=['example_1.jpg', 'example_2.jpg', 'example_3.jpg'], inputs=[img_input])

            with gr.Column(scale=0.5):
                img_output = gr.Plot()

        with gr.Row():
            chatbot = gr.Chatbot()
        
        with gr.Row():            
            txt_input = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)

        # define action
        btn_analysis.click(fn = predict_image_func, 
                            inputs = [img_input, state],
                            outputs = [img_output, chatbot, state])
        txt_input.submit(fn = predict_chat_func, 
                         inputs = [txt_input, state], 
                         outputs = [chatbot, state])
        txt_input.submit(lambda :"", None, txt_input)

    demo.launch(server_name="0.0.0.0", debug=True)

def predict():
    return [predict_image, predict_chat, gui_interface]

# 모델 테스트

In [84]:
import os

os.environ["API_ADDRESS"] = 'https://0.0.0.0'
os.environ["DEMOAPI_ID"] = '0000'

import requests

api_env = os.environ['API_ADDRESS']
demo_id = os.environ['DEMOAPI_ID']

def sendResultForDemoAPI(error_msg):
    res = requests.post(api_env, json= {'dtype': 2, 'id': demo_id, 'error': error_msg})
    return

def sendResultForDemoAPIWithID(demoid, error_msg):
    res = requests.post(api_env, json= {'dtype': 2, 'id': demo_id, 'error': error_msg})
    return

def demo_from_submission(key, pyname, func):

    try:
        _predict_image_func, _predict_chat_func, _gui_interface_func = func()

        _gui_interface_func(_predict_image_func, _predict_chat_func)

    except Exception as e:
        sendResultForDemoAPI(str(e))

if __name__ == "__main__":
    demo_from_submission("","", predict)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/tmp/tmpfw64ho5_.png
Keyboard interruption in main thread... closing server.


''

# 모델 제출

AIFactory 플랫폼을 통해서 개발한 모델을 쉽게 배포하여 서비스를 구동할 수 있습니다. 이 서비스를 이용하기 위해서는 AIFactory에 태스크를 등록한 후 모델을 제출해야하는데요. 아래 코드가 AIFactory에 제출하는 코드입니다. 더 자세한 것이 궁금하시다면 contact@aifactory.page로 연락주세요.

In [86]:
'''
import aifactory.grade as aif
import ipynbname
import os

if __name__ == "__main__":  
    filename = ''
    try: 
        filename = ipynbname.name()
    except Exception as e:
        filename = os.path.basename(__file__)
    print(filename)
    aif.submit("AIFactory 제출 키", filename, predict)
'''    

fileId=1ZJk-7OOEpHyD0klGQ8xtl3tq-3vn0gmM
Running on CoLab
google colab


Downloading...
From: https://drive.google.com/uc?id=1ZJk-7OOEpHyD0klGQ8xtl3tq-3vn0gmM
To: /content/task.ipynb
100%|██████████| 27.1k/27.1k [00:00<00:00, 11.9MB/s]


success
